## If you liked this Project then you can find more at https://pankajshah.netlify.app
#### Feel free to contact me if you have any suggestions

In [ ]:
!pip install tqdm

In [ ]:
from google.colab import files
import requests
from tqdm import tqdm
import pandas as pd
from bs4 import BeautifulSoup

d = {'Title': [], 'Link': []}

pages = int(input('Enter Pages:'))
for i in tqdm(range(1, pages+1), desc="Pages", unit="page"):
    url = f'https://www.hdhub4u.tw/page{i}'
    response = requests.get(url)

    if response.status_code != 200:
        print(f"Failed to retrieve page {i}. Skipping...")
        continue

    soup = BeautifulSoup(response.content, 'html.parser')
    recent_movies = soup.find('ul', class_='recent-movies')

    if recent_movies:
        for link in recent_movies.find_all('a', href=True):
            title = link.get_text(strip=True)
            href = link['href']

            if title and href and href not in d['Link']:
                d['Title'].append(title)
                d['Link'].append(href)
    else:
        print(f"Page {i}: The <ul> with class 'recent-movies' was not found.")

d_count_ofLinks = len(d['Link'])
print(f'Found {d_count_ofLinks} movies in {pages} pages')
print()
final_data = {}

i = 0

with tqdm(total=d_count_ofLinks, desc="Processing Links", unit="link") as pbar:
    while True:
        if i >= d_count_ofLinks:
            break

        next_link = d['Link'][i]

        try:
            new_response = requests.get(next_link)
        except requests.exceptions.RequestException as e:
            print(f"Request failed for link {i+1}: {e}")
            i += 1
            pbar.update(1)
            continue

        if new_response.status_code != 200:
            print(f"Failed to retrieve page {i+1}. Skipping...")
            i += 1
            pbar.update(1)
            continue

        soup = BeautifulSoup(new_response.content, 'html.parser')

        kp_header_div = soup.find('div', class_='Z1hOCe')
        first_h2_text = None
        if kp_header_div:
            first_h2 = kp_header_div.find('h2')
            if first_h2:
                first_h2_text = first_h2.get_text(strip=True)
        else:
            print("We Found Some Error (FIX IT)")

        link_element = soup.find(
            'div',
            style='text-align: center;',
            attrs={'data-ved': '2ahUKEwi0gOTl-ozlAhWfILcAHVY0DbIQyxMoADAvegQIERAJ'}
        )

        if link_element:
            links = link_element.find_all('a')
            link_data = []

            for link in links:
                href = link.get('href')
                text = link.get_text(strip=True)
                link_data.append((text, href))

            if first_h2_text:
                final_data[first_h2_text] = link_data

        i += 1
        pbar.update(1)

data_for_df = []

for h2_text, links in final_data.items():
    for link_text, link_url in links:
        data_for_df.append({
            'Movie Name': h2_text,
            'Quality': link_text,
            'URL': link_url
        })

df = pd.DataFrame(data_for_df)
df.to_csv('Movies_Link.csv', index=False)
print()
print('Link Files Saved')
files.download('Movies_Link.csv')
